In [1]:
!cp -r /content/drive/MyDrive/DERMNET/AnnotateDataset.zip ./

In [ ]:
!unzip AnnotateDataset.zip

In [1]:
!pip install -q tflite-model-maker
!pip install -q tflite-support

In [2]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

In [3]:
spec = model_spec.get('efficientdet_lite4')

In [4]:
import glob
import shutil
import os

src_dir = "Dataset/train/**"
dst_dir = "Dataset/train"
for jpgfile in glob.iglob(os.path.join(src_dir, "*.jpg")):
    shutil.move(jpgfile, dst_dir)

In [7]:
import glob
import shutil
import os

src_dir = "Dataset/train/**"
dst_dir = "Dataset/train"
for jpgfile in glob.iglob(os.path.join(src_dir, "*.xml")):
    shutil.move(jpgfile, dst_dir)

In [8]:
import glob
import shutil
import os

src_dir = "Dataset/val/**"
dst_dir = "Dataset/val"
for jpgfile in glob.iglob(os.path.join(src_dir, "*.jpg")):
    shutil.move(jpgfile, dst_dir)

In [9]:
import glob
import shutil
import os

src_dir = "Dataset/val/**"
dst_dir = "Dataset/val"
for jpgfile in glob.iglob(os.path.join(src_dir, "*.xml")):
    shutil.move(jpgfile, dst_dir)

In [10]:
for i in ["Acne", "Herpes", "Melanoma", "NailFungus", "Psoraisis", "Scabies", "VascularTumor", "Wartz"]:
    os.rmdir(f"Dataset/train/{i}")
    os.rmdir(f"Dataset/val/{i}")

In [5]:
train_data = object_detector.DataLoader.from_pascal_voc(
    'Dataset/train',
    'Dataset/train',
    ['VascularTumor','Wartz','Psoraisis','NailFungus','Acne','Melanoma','Scabies','Herpes']
)

val_data = object_detector.DataLoader.from_pascal_voc(
    'Dataset/val',
    'Dataset/val',
    ['VascularTumor','Wartz','Psoraisis','NailFungus','Acne','Melanoma','Scabies','Herpes']
)

In [ ]:
model = object_detector.create(train_data, model_spec=spec, batch_size=4, train_whole_model=True, epochs=10, validation_data=val_data)

Epoch 1/10
1786/1786 [==============================] - 1681s 906ms/step - det_loss: 1.1478 - cls_loss: 0.7645 - box_loss: 0.0077 - reg_l2_loss: 0.1106 - loss: 1.2584 - learning_rate: 0.0065 - gradient_norm: 4.4038 - val_det_loss: 0.9490 - val_cls_loss: 0.6014 - val_box_loss: 0.0070 - val_reg_l2_loss: 0.1113 - val_loss: 1.0604
Epoch 2/10
1786/1786 [==============================] - 1617s 906ms/step - det_loss: 0.9495 - cls_loss: 0.6207 - box_loss: 0.0066 - reg_l2_loss: 0.1116 - loss: 1.0612 - learning_rate: 0.0047 - gradient_norm: 4.1021 - val_det_loss: 0.8098 - val_cls_loss: 0.5550 - val_box_loss: 0.0051 - val_reg_l2_loss: 0.1119 - val_loss: 0.9217
Epoch 3/10
1786/1786 [==============================] - 1618s 906ms/step - det_loss: 0.8929 - cls_loss: 0.5816 - box_loss: 0.0062 - reg_l2_loss: 0.1121 - loss: 1.0051 - learning_rate: 0.0041 - gradient_norm: 4.2496 - val_det_loss: 0.8821 - val_cls_loss: 0.5568 - val_box_loss: 0.0065 - val_reg_l2_loss: 0.1124 - val_loss: 0.9945
Epoch 4/10
17

In [ ]:
model.evaluate(val_data)

In [ ]:
model.export(export_dir='.', tflite_filename='android.tflite')

In [ ]:
model.evaluate_tflite('android.tflite', val_data)